<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/CollabNotebook/TASC_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!unzip Corrected_data.zip


Archive:  Corrected_data.zip
   creating: Corrected_data/
  inflating: Corrected_data/BombBlast(categorised).json  
  inflating: Corrected_data/Earthquake(categorised).json  
  inflating: Corrected_data/flood(categorised).json  
  inflating: Corrected_data/tornado(categorised).json  
  inflating: Corrected_data/tsnuami (categorised).json  
  inflating: Corrected_data/Volcano(categorised).json  
  inflating: Corrected_data/war(categorised).json  


In [5]:
import json
import os

all_data = []
for filename in os.listdir('Corrected_data/'):
    if filename.endswith(".json"):
        file_path = os.path.join('Corrected_data/', filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            all_data.extend(data)

with open("Data.json", 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

All JSON files have been combined into 'Data.json'


In [7]:
import pandas as pd

data=pd.read_json("Data.json")

In [10]:
data["intent"].unique()

array(['Miscellaneous', 'floods', 'Miscellenous', 'Miscellaenous',
       'Miscellneous', 'BombBlast', 'Micscellaneous', 'Micsellaneous',
       'War', 'Micellaneous', 'miscellaneous', 'Miscellaeous',
       'Earthquake', nan, 'Volcano', 'Tornado', 'Miscellanous',
       'Miscelleneous'], dtype=object)